<a href="https://colab.research.google.com/github/govindaa-sharma/MLPrcatice/blob/main/Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [28]:
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset = pd.read_csv('/content/extracted_folder/IMDB Dataset.csv')

In [ ]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
dataset['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [ ]:
encoder = LabelEncoder()
dataset['sentiment'] = encoder.fit_transform(dataset['sentiment'])

In [ ]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
train_text, test_text, train_labels, test_labels = train_test_split(dataset['review'].tolist(), dataset['sentiment'].tolist(), test_size=0.2, random_state=42)

In [ ]:
def tokenize(text):
  return tokenizer(text, padding='max_length', truncation=True, max_length=128)

In [ ]:
train_encoded = tokenize(train_text)
test_encoded = tokenize(test_text)

In [26]:
len(train_encoded)

3

In [29]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encoded),
    train_labels
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encoded),
    test_labels
))

train_dataset = train_dataset.shuffle(1000).batch(16).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(16).prefetch(tf.data.AUTOTUNE)

In [30]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [31]:
history = model.fit(train_dataset, validation_data=test_dataset, epochs=2)

Epoch 1/2
2500/2500 [==============================] - 1242s 477ms/step - loss: 0.3214 - accuracy: 0.8562 - val_loss: 0.2697 - val_accuracy: 0.8837
Epoch 2/2
2500/2500 [==============================] - 1190s 476ms/step - loss: 0.1806 - accuracy: 0.9288 - val_loss: 0.3129 - val_accuracy: 0.8862


In [32]:
loss, acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {acc:.2f}")


625/625 [==============================] - 93s 149ms/step - loss: 0.3129 - accuracy: 0.8862
Test Accuracy: 0.89


In [38]:
# Predict example
sample = "Terrible movie. The dialogue was cringeworthy, and the plot made no sense. I honestly can't believe it got released."
tokens = tokenizer(sample, return_tensors="tf", truncation=True, padding=True)
logits = model(tokens)[0]
predicted_label = tf.argmax(logits, axis=1).numpy()[0]
print("Predicted Sentiment:", "Positive" if predicted_label == 1 else "Negative")


Predicted Sentiment: Negative
